In [1]:
import os
from mpi4py import MPI
from petsc4py import PETSc
import numpy as np
import ufl
from basix.ufl import element, mixed_element
from dolfinx import mesh, fem
from dolfinx import default_real_type, log, plot
from dolfinx.fem import Function, functionspace
from dolfinx.fem.petsc import NonlinearProblem
from dolfinx.io import XDMFFile
from dolfinx.mesh import CellType, create_unit_square
from dolfinx.nls.petsc import NewtonSolver
from ufl import dx, grad, inner


try:
    import pyvista as pv
    import pyvistaqt as pvqt
    have_pyvista = True
    if pv.OFF_SCREEN:
        pv.start_xvfb(wait=0.5)
except ModuleNotFoundError:
    print("pyvista and pyvistaqt are required to visualize the solution")
    have_pyvista = False

# Save all logging to file
log.set_output_file("log.txt")



pyvista and pyvistaqt are required to visualize the solution


In [2]:
lmbda = 1.0  # surface parameter
t = 0.0    # start time
T = 0.5    # end time
n = 10000   # number of steps 
dt = (T-t)/n
theta = 0.5      # time-stepping method

#msh = create_unit_square(MPI.COMM_WORLD, 96, 96, CellType.triangle)
#msh.geometry.x[:] *= np.pi
# Define mesh
nx, ny = 1000, 1000
msh = mesh.create_rectangle(MPI.COMM_WORLD, [np.array([0.0, 0.0]), np.array([np.pi, np.pi])],
                               [nx, ny], mesh.CellType.triangle)




P1 = element("Lagrange", msh.basix_cell(), 1)#, dtype=default_real_type)
ME = functionspace(msh, mixed_element([P1, P1]))
print("Mixed Element Components:")

# Define test functions and unknowns
q, v = ufl.TestFunctions(ME)
u = Function(ME)  # current solution
u0 = Function(ME)  # solution from previous converged step
print("Functions u and u0 initialized.")

# Split mixed functions
c, mu = ufl.split(u)
c0, mu0 = ufl.split(u0)

# Zero u
u.x.array[:] = 0.0
print("Initial solution u set to zero.")

Mixed Element Components:
Functions u and u0 initialized.
Initial solution u set to zero.


In [3]:
# Define exact initial condition
class exact_solution:
    def __init__(self,t):
        self.t=t
    def __call__(self,x):
        return -2.0*np.exp(-4 * t) * np.cos(x[0]) * np.cos(x[1])

u_exact = exact_solution(t)
# Set initial condition using exact solution at t = 0
u.sub(0).interpolate(u_exact)
u.x.scatter_forward()
u0.sub(0).interpolate(u_exact)
u0.x.scatter_forward()


# Define forcing expression
class ForcingExpression:
    def __init__(self, lmbda, t):
        self.lmbda = lmbda
        self.t = t

    def __call__(self, x):
        result = 0.0
       
        # Normalize the result to a range [0, 1] (or any other desired range)
#         min_val = np.min(result)  # Find the minimum value in the result
#         max_val = np.max(result)  # Find the maximum value in the result
#         normalized_result = (result - min_val) / (max_val - min_val)  # Normalize to [0, 1]

        return result  # Return normalized result

# Function to compute the L2 error
def compute_l2_error(u_num, t):
    u_exact = Function(ME)
    u_exact.sub(0).interpolate(exact_solution(t))

    # # Print array shapes to debug the mismatch
    # print(f"Shape of numerical solution array: {u_num.x.array.shape}")
    # print(f"Shape of exact solution array: {u_exact.x.array.shape}")

    # Check if the shapes match
    assert u_num.x.array.shape == u_exact.x.array.shape, \
        "Shape mismatch between numerical and exact solution arrays."
    
    # Use the UFL representation to calculate the error
    error_expr = u_num - u_exact

    # Compute the L2 norm of the error
    L2_error_form = fem.form(inner(error_expr, error_expr) * dx)
    L2_error = fem.assemble_scalar(L2_error_form)

    return np.sqrt(L2_error)

In [4]:
# Compute the chemical potential df/dc
c = ufl.variable(c)
f = 0.0#0.25*(1-c**2)**2
dfdc = ufl.diff(f, c)
print("Chemical potential computed.")
forcing_expr= ForcingExpression(lmbda, t)
V0, dofs = ME.sub(0).collapse()
#forcing_term= Function(V0)
#forcing_term.interpolate(lambda x: forcing_expr(x))
print("Forcing function interpolated successfully.")


mu_mid = (1.0 - theta) * mu0 + theta * mu
# Weak statement of the equations
F0 = inner(c, q) * dx - inner(c0, q) * dx + dt * inner(grad(mu_mid), grad(q)) * dx #- inner(forcing_term, q) * dx
F1 = inner(mu, v) * dx - lmbda * inner(grad(c), grad(v)) * dx #- inner(dfdc, v) * dx

F = F0 + F1
print("Variational forms defined.")


Chemical potential computed.
Forcing function interpolated successfully.
Variational forms defined.


In [5]:
# Create nonlinear problem and Newton solver
problem = NonlinearProblem(F, u)
solver = NewtonSolver(MPI.COMM_WORLD, problem)
solver.convergence_criterion = "residual"
solver.rtol = np.sqrt(np.finfo(float).eps) * 1e-2
print("Nonlinear problem and solver created.")

# Set PETSc options for the Krylov solver
ksp = solver.krylov_solver
opts = PETSc.Options()
option_prefix = ksp.getOptionsPrefix()
opts[f"{option_prefix}ksp_type"] = "preonly"
opts[f"{option_prefix}pc_type"] = "lu"
sys = PETSc.Sys()
if sys.hasExternalPackage("mumps"):
    opts[f"{option_prefix}pc_factor_mat_solver_type"] = "mumps"
elif sys.hasExternalPackage("superlu_dist"):
    opts[f"{option_prefix}pc_factor_mat_solver_type"] = "superlu_dist"
ksp.setFromOptions()
print("Krylov solver options set.")

Nonlinear problem and solver created.
Krylov solver options set.


In [ ]:
# Output file
# file = XDMFFile(MPI.COMM_WORLD, "demo_ch/output.xdmf", "w")
# file.write_mesh(msh)
# print("Mesh written to output file.")


# Pyvista visualization
if have_pyvista:
    topology, cell_types, x = plot.vtk_mesh(V0)
    grid = pv.UnstructuredGrid(topology, cell_types, x)
    grid.point_data["c"] = u.x.array[dofs].real
    grid.set_active_scalars("c")
    p = pvqt.BackgroundPlotter(title="concentration", auto_update=True)
    max_val = np.max(np.abs(u.x.array[dofs].real))
    p.add_mesh(grid, clim=[-max_val, max_val])
    p.view_xy(True)
    p.add_text(f"time: {t}", font_size=12, name="timelabel")

c = u.sub(0)
u0.x.array[:] = u.x.array
while t < T:
    t += dt

    # Update the forcing term
#     forcing_expr.t = t
#     forcing_term.interpolate(lambda x: forcing_expr(x))
#     forcing_term.x.scatter_forward()

    r = solver.solve(u)
    print(f"Solve result: {r}")

    # Print the number of iterations for the current time step
    print(f"Step {int(t / dt)}: num iterations: {r[0]}")

    # Update the previous solution u0 with the current solution
    u0.x.array[:] = u.x.array

    
    # # Write the solution to file for post-processing
    print(f"Shape of ccc numerical solution array: {u.x.array.shape}")
    L2_error = compute_l2_error(u, t)
    print(f"L2 error at time {t}: {L2_error}")
    #file.write_function(c, t)

    # Update visualization (if applicable)
    if have_pyvista:
        p.add_text(f"time: {t:.2e}", font_size=12, name="timelabel")
        grid.point_data["c"] = u.x.array[dofs].real
        p.app.processEvents()

#file.close()

Solve result: (1, True)
Step 1: num iterations: 1
Shape of ccc numerical solution array: (2004002,)
L2 error at time 5e-05: 6.2825622235858205


In [ ]:
# Update ghost entries and plot
if have_pyvista:
    u.x.scatter_forward()
    grid.point_data["c"] = u.x.array[dofs].real
    screenshot = None
    if pv.OFF_SCREEN:
        screenshot = "c.png"
    pv.plot(grid, show_edges=True, screenshot=screenshot)